## Test for Liquidity Leaks

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import time
import datetime
import math
import matplotlib.pyplot as plt

In [2]:
from uniswappy.math.model import TokenDeltaModel
from uniswappy.math.model import EventSelectionModel
from uniswappy.cpt.factory import Factory
from uniswappy.cpt.exchg import Exchange
from uniswappy.erc import ERC20
from uniswappy.process.deposit import SwapDeposit
from uniswappy.process.swap import WithdrawSwap
from uniswappy.process.liquidity import RemoveLiquidity
from uniswappy.process.liquidity import AddLiquidity
from uniswappy.process.swap import Swap
from uniswappy.cpt.quote import LPQuote

In [3]:
user_nm = 'user0'
dai_amount = 10000
sys_amount = 100000

In [4]:
dai1 = ERC20("DAI", "0x111")
sys1 = ERC20("SYS", "0x09")

factory = Factory("SYS pool factory", "0x2")
lp = factory.create_exchange(sys1, dai1, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, 1000, 1000, 1000, 1000)
lp.info()
print('lp tokens: {:.6f} sqrt(k): {:.6f}\n'.format(lp.total_supply, np.sqrt(lp.reserve0*lp.reserve1)))

amt = 100

print('-----------------------------------------')
print('1 LP token is worth {:.6f} DAI'.format(LPQuote(False).get_amount_from_lp(lp, dai1, 1)))
print('-----------------------------------------\n')

for k in range(10):
    
    RemoveLiquidity().apply(lp, dai1, user_nm, amt) 
    lp.info()
    print('(sub liquidity) lp tokens: {:.6f} sqrt(k) {:.6f}\n'.format(lp.total_supply, np.sqrt(lp.reserve0*lp.reserve1)))    
    
    AddLiquidity().apply(lp, dai1, user_nm, amt) 
    lp.info()
    print('(add liquidity) lp tokens: {:.6f} sqrt(k): {:.6f}'.format(lp.total_supply, np.sqrt(lp.reserve0*lp.reserve1)))


    print('-----------------------------------------\n')
    
print('-----------------------------------------')
print('1 LP token is worth {:.6f} DAI'.format(LPQuote(False).get_amount_from_lp(lp, dai1, 1)))
print('-----------------------------------------\n')    

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS = 1000 | DAI = 1000
Liquidity: 1000.0 

lp tokens: 1000.000000 sqrt(k): 1000.000000

-----------------------------------------
1 LP token is worth 1.996006 DAI
-----------------------------------------

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS = 900.0 | DAI = 900.0
Liquidity: 900.0 

(sub liquidity) lp tokens: 900.000000 sqrt(k) 900.000000

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS = 1000.0 | DAI = 1000.0
Liquidity: 1000.0 

(add liquidity) lp tokens: 1000.000000 sqrt(k): 1000.000000
-----------------------------------------

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS = 900.0 | DAI = 900.0
Liquidity: 900.0 

(sub liquidity) lp tokens: 900.000000 sqrt(k) 900.000000

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS = 1000.0 | DAI = 1000.0
Liquidity: 1000.0 

(add liquidity) lp tokens: 1000.000000 sqrt(k): 1000.000000
-----------------------------------------

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS

In [5]:
dai1 = ERC20("DAI", "0x111")
sys1 = ERC20("SYS", "0x09")


factory = Factory("SYS pool factory", "0x2")
lp = factory.create_exchange(sys1, dai1, symbol="LP", address="0x011")
lp.add_liquidity(user_nm, sys_amount, dai_amount, sys_amount, dai_amount)
lp.info()

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS = 100000 | DAI = 10000
Liquidity: 31622.776601683792 



### Liquidity leak test-batch #1

In [6]:
lp.info()
dai_amt = LPQuote(False).get_amount_from_lp(lp, dai1, 1)
print('Pre-tests:')
print('- 1 LP token is {:.6f} DAI'.format(dai_amt))
print('- we are starting with {:.6f} LP token'.format(lp.liquidity_providers[user_nm]))

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS = 100000 | DAI = 10000
Liquidity: 31622.776601683792 

Pre-tests:
- 1 LP token is 0.631497 DAI
- we are starting with 31622.776602 LP token


#### Test 1: AddLiquidity / RemoveLiquidity

In [7]:
for k in range(10):
    vault_lp1_amt = 2000
    dai_amount1 = LPQuote(True).get_amount(lp, dai1, vault_lp1_amt) 
    AddLiquidity().apply(lp, dai1, user_nm, vault_lp1_amt) 
    RemoveLiquidity().apply(lp, dai1, user_nm, abs(vault_lp1_amt))  

In [8]:
lp.info()
dai_amt = LPQuote(False).get_amount_from_lp(lp, dai1, 1)
print('Post test:')
print('- 1 LP token is {:.6f} DAI'.format(dai_amt))
print('- we have {:.6f} LP token remaining'.format(lp.liquidity_providers[user_nm]))

Exchange SYS/DAI (LP)
Coins: SYS/DAI
Reserves: SYS = 100000.0 | DAI = 10000.0
Liquidity: 31622.776601683792 

Post test:
- 1 LP token is 0.631497 DAI
- we have 31622.776602 LP token remaining
